In [17]:
%pip install --upgrade --quiet mistralai

Note: you may need to restart the kernel to use updated packages.


In [18]:
%pip install langchain-mistralai

Note: you may need to restart the kernel to use updated packages.


In [19]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [20]:
api_key = os.getenv("MISTRAL_API_KEY")
api_key

'0lzefmPJKlVKaL8hWROiGjWV22P1itTN'

In [21]:
# from mistralai import Mistral

# model = "ministral-8b-latest"

# client = Mistral(api_key=api_key)

# chat_response = client.chat.complete(
#     model= model,
#     messages = [
#         {
#             "role": "user",
#             "content": "What is the best French cheese?",
#         },
#     ]
# )
# print(chat_response.choices[0].message.content)

In [22]:
from langchain_mistralai import ChatMistralAI
from langchain_core.messages import SystemMessage, HumanMessage

## 

In [23]:
llm = ChatMistralAI(
    model="mistral-small-latest",
    temperature=0.3,
    max_retries=5
)

# Prompts
sys_prompt = "You are a helpful assistant that translates English to Spanish"
hum_prompt = "Translate the user sentence without giving more details, just the translation: I love programming."

# Invoke LLM
response = llm.invoke([
    SystemMessage(content=sys_prompt),
    HumanMessage(content=hum_prompt)]
).content.strip()

print(response)

Me encanta programar.


## structuring agent

In [42]:
import os
from typing import Dict, List, Any
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_mistralai import ChatMistralAI
from langgraph.graph import StateGraph, END
from pydantic import BaseModel, Field

# Define our state
class AgentState(BaseModel):
    messages: List[Any] = Field(default_factory=list)
    formatted_response: str = ""

# Initialize MistralAI LLM
def initialize_llm(api_key=None):
    if api_key:
        os.environ["MISTRAL_API_KEY"] = api_key
    
    # Make sure MISTRAL_API_KEY is set in environment variables if not passed as parameter
    return ChatMistralAI(
        model="mistral-small-latest",  # You can change the model as needed
        temperature=0.1  # Low temperature for more deterministic responses
    )

# Define the system prompt that contains formatting instructions
def create_system_prompt(formatting_instructions):
    return SystemMessage(content=f"""
You are a query rewriter agent that takes raw user and rewrites the query in a concise manner so that the information in the following fields are highlighted:

{formatting_instructions}

Just give the rewritten query. Do not include any explanations or additional text in your response.
Just provide the rewritten result.
""")

# Node to format the query
def format_query(state: AgentState) -> AgentState:
    llm = initialize_llm()
    
    # Get the most recent user message
    user_message = state.messages[-1]
    
    # Include system message and user message for the LLM call
    system_message = state.messages[0]
    messages = [system_message, user_message]
    
    # Get formatted response from LLM
    response = llm.invoke(messages)
    
    # Update state with formatted response
    state.formatted_response = response.content
    return state

# Node to prepare final response
def prepare_response(state: AgentState) -> AgentState:
    # Simply pass through the formatted response
    return state

# Create the graph
def create_formatting_agent(formatting_instructions):
    # Create system prompt
    system_prompt = create_system_prompt(formatting_instructions)
    
    # Define the graph
    workflow = StateGraph(AgentState)
    
    # Add nodes
    workflow.add_node("format_query", format_query)
    workflow.add_node("prepare_response", prepare_response)
    
    # Add edges
    workflow.add_edge("format_query", "prepare_response")
    workflow.add_edge("prepare_response", END)
    
    # Set entry point
    workflow.set_entry_point("format_query")
    
    # Compile the graph
    agent = workflow.compile()
    
    # Return a function that initializes the state and runs the agent
    def run_agent(query):
        messages = [system_prompt, HumanMessage(content=query)]
        result = agent.invoke({"messages": messages})
        return result
    
    return run_agent

In [45]:

# Define your formatting instructions
formatting_instructions = """
Fields : Car Company, Car model, Year of production, Mileage, Price, Color, Type of fuel, Engine Transmission, Engine Capacity
"""

# Create the agent
formatting_agent = create_formatting_agent(formatting_instructions)

# Test with a sample query
query = "I'm a student. I dont earn that much. Suggest me some cars that I can buy"
result = formatting_agent(query)
print(result['formatted_response'])

Suggest some affordable cars.


## grading agent


In [73]:
import os
from typing import Dict, List, Any, Optional
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_mistralai import ChatMistralAI
from langgraph.graph import StateGraph, END
from pydantic import BaseModel, Field

# Define our state
class GradingAgentState(BaseModel):
    messages: List[Any] = Field(default_factory=list)
    query: str = ""
    results: List[str] = Field(default_factory=list)
    graded_results: List[Dict[str, Any]] = Field(default_factory=list)

# Initialize MistralAI LLM
def initialize_llm(api_key=None):
    if api_key:
        os.environ["MISTRAL_API_KEY"] = api_key
    
    # Make sure MISTRAL_API_KEY is set in environment variables if not passed as parameter
    return ChatMistralAI(
        model="mistral-large-latest",  # You can change the model as needed
        temperature=0.1  # Low temperature for more deterministic grading
    )

# Define the system prompt for the grading agent
def create_system_prompt():
    return SystemMessage(content="""
You are a grading agent that evaluates the accuracy of search results based on a user query.

For each result, you will assign one of the following grades:
- 2: Accurate - The result directly and correctly answers the query
- 1: Moderately Accurate - The result is partially relevant or contains some useful information related to the query
- 0: Inaccurate - The result is irrelevant, contains incorrect information, or doesn't address the query

Your response should be formatted as follows:
```
[
  {"result_index": 0, "grade": <2|1|0>},
  {"result_index": 1, "grade": <2|1|0>},
  ...
]
```

Do not include any additional text, commentary, or explanations outside this JSON structure.
Focus solely on the accuracy and relevancy of each result in relation to the query.
""")

# Node to process the input query and results
def process_input(state: GradingAgentState) -> GradingAgentState:
    # The query and results are already set in the state from the run_agent function
    return state

# Node to grade the results
def grade_results(state: GradingAgentState) -> GradingAgentState:
    llm = initialize_llm()
    
    # If there are no results, return empty graded_results
    if not state.results:
        state.graded_results = []
        return state
    
    # Prepare the message for the LLM
    system_message = create_system_prompt()
    
    prompt_content = f"""
Query: {state.query}

Results to grade:
"""
    
    for i, result in enumerate(state.results):
        prompt_content += f"\nResult {i}:\n{result}\n"
    
    prompt_message = HumanMessage(content=prompt_content)
    
    # Get grading from LLM
    response = llm.invoke([system_message, prompt_message])
    
    # Parse the response to get the graded results
    try:
        # Extract JSON from the response
        response_content = response.content
        
        # If the response is wrapped in code blocks, extract the content
        if "```" in response_content:
            response_content = response_content.split("```")[1]
            if response_content.startswith("json"):
                response_content = response_content[4:].strip()
        
        import json
        graded_results = json.loads(response_content)
        state.graded_results = graded_results
    except Exception as e:
        # If parsing fails, create a fallback response
        state.graded_results = [
            {"result_index": i, "grade": 0, "explanation": "Failed to parse grading result"}
            for i in range(len(state.results))
        ]
    
    return state

# Node to format the final response
def format_response(state: GradingAgentState) -> GradingAgentState:
    # The state already contains the graded results, so we can return it as is
    return state

# Create the grading agent
def create_grading_agent():
    # Define the graph
    workflow = StateGraph(GradingAgentState)
    
    # Add nodes
    workflow.add_node("process_input", process_input)
    workflow.add_node("grade_results", grade_results)
    workflow.add_node("format_response", format_response)
    
    # Add edges
    workflow.add_edge("process_input", "grade_results")
    workflow.add_edge("grade_results", "format_response")
    workflow.add_edge("format_response", END)
    
    # Set entry point
    workflow.set_entry_point("process_input")
    
    # Compile the graph
    agent = workflow.compile()
    
    # Return a function that takes query and results separately
    def run_agent(query: str, results: List[str]):
        system_prompt = create_system_prompt()
        
        # Initialize messages with system prompt
        messages = [system_prompt]
        print(f"")
        
        # Create initial state with query and results
        initial_state = GradingAgentState(
            messages=messages,
            query=query,
            results=results
        )
        
        # Invoke the agent with the initial state
        result = agent.invoke(initial_state)
        grades_arr = result['graded_results']
        graded_query_results = []
        for i in range(len(grades_arr)):
            obj = {
                "query" : query,
                "result" : results[int(grades_arr[i]['result_index'])],
                "grade" : grades_arr[i]['grade']
            }
            graded_query_results.append(obj)
        
        return graded_query_results
    
    return run_agent


In [74]:

grading_agent = create_grading_agent()
    
# Example query and results
query = "What is the capital of France?"
results = [
        "Paris is the capital and most populous city of France.",
        "The capital of Italy is Rome, which is known as the Eternal City.",
        "France's capital city is Paris, known for the Eiffel Tower and Louvre Museum."
    ]

    
# Run the agent
graded_results = grading_agent(query, results)

# Display the results
print("Grading Results:")
for result in graded_results:
    # grade_text = {2: "Accurate", 1: "Moderately Accurate", 0: "Inaccurate"}[result["grade"]]
    print(result, '\n')


Grading Results:
{'query': 'What is the capital of France?', 'result': 'Paris is the capital and most populous city of France.', 'grade': 2} 

{'query': 'What is the capital of France?', 'result': 'The capital of Italy is Rome, which is known as the Eternal City.', 'grade': 0} 

{'query': 'What is the capital of France?', 'result': "France's capital city is Paris, known for the Eiffel Tower and Louvre Museum.", 'grade': 2} 

